In [1]:
import os
import google.generativeai as genai
import json
from dotenv import load_dotenv

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.10) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
# Load environment variables .env
load_dotenv()

# Get the Gemini API key from .env
api_key = os.getenv('GEMINI_API_KEY')

In [3]:
# Configure Gemini with API key
genai.configure(api_key=api_key)

# Debug: Print available models
print("Available models:", [m.name for m in genai.list_models()])

# Create the model
model = genai.GenerativeModel('models/gemini-pro-latest')


Available models: ['models/embedding-gecko-001', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash', 'models/gemini-2.5-pro-preview-05-06', 'models/gemini-2.5-pro-preview-06-05', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.0-flash-thinking-exp-01-21', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp-1219', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/learnlm-2.0-flash-experimental', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'mo

In [4]:
class AccountSummaryAgent:
    def __init__(self, model_name="gemini-2.5-flash"):
        self.model = genai.GenerativeModel(model_name)
 
    def create_summary(self, account_data: dict, health_score: float = None) -> str:
        """
        Generate an executive-level account summary from account data.
        
        Args:
            account_data (dict): Account information including:
                - account_name: Name of the account
                - industry/sector: Industry vertical
                - annual_revenue: Annual revenue
                - recent_activity: List of recent activities
                - open_opportunities: List of open opportunities
                - risk_factors: List of identified risks
                - Any other relevant account details
            health_score (float): Optional health score (0-1 scale)
            
        Returns:
            str: Generated account summary with analysis and recommendations
        """
        
        # ✅ FIX: Format health score BEFORE using it in the f-string
        if health_score is not None:
            health_score_str = f"{health_score:.2f}"
        else:
            health_score_str = "Not available"
        
        prompt = f"""
You are an expert CRM account analyst.

You will be given structured account data and a predictive health score.
Return a concise, executive-level account summary.

Account Health Score: {health_score_str}

Account Data (JSON):
{json.dumps(account_data, indent=2)}

Generate the following sections:

1. **Executive Summary** (3-4 sentences)
2. **Key Strengths** (bullet points)
3. **Key Risks or Warning Signs** (bullet points)
4. **Open Opportunities** (bullet points if any)
5. **Recommended Next Actions** (3 concrete steps with owners + timelines)
6. **Overall Priority Level** (High / Medium / Low)

Be specific, data-driven, and professional.
"""
        
        response = self.model.generate_content(prompt)
        return response.text

In [5]:
# ------- TEST THE ACCOUNT SUMMARY AGENT -------
 
# Sample account data for testing
test_account_data = {
    "account_name": "Acme Corp",
    "industry": "FinTech",
    "annual_revenue": 8500000,
    "recent_activity": ["QBR scheduled", "Support ticket escalated"],
    "open_opportunities": ["Renewal expansion", "Cross-sell analytics package"],
    "risk_factors": ["Reduced logins", "Competitor outreach noticed"]
}
 
# Create an instance of the agent
agent = AccountSummaryAgent(model_name="gemini-2.5-flash")
 
# Generate the summary
output = agent.create_summary(test_account_data, health_score=0.72)
 
# Print the result
print("=== ACCOUNT SUMMARY ===\n")
print(output)
print("\n========================")
 

=== ACCOUNT SUMMARY ===

Here is an executive-level account summary for Acme Corp:

---

**Account Summary: Acme Corp**

**1. Executive Summary**
Acme Corp, a high-value FinTech client with $8.5M annual revenue, currently holds a health score of 0.72, indicating a need for proactive engagement. While a QBR is scheduled and significant expansion opportunities exist, recent critical warning signs, including reduced user logins and competitor outreach, demand immediate attention. An escalated support ticket further highlights potential satisfaction issues that could impact retention and growth.

**2. Key Strengths**
*   **Significant Annual Revenue:** $8.5M, establishing Acme Corp as a top-tier client.
*   **Proactive Engagement:** A QBR is scheduled, providing a platform for strategic review and relationship management.
*   **Growth Potential:** Active opportunities for both renewal expansion and a cross-sell of the analytics package.

**3. Key Risks or Warning Signs**
*   **Usage Declin